In [40]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import * 
from qiskit_research.utils.convenience import *
import stim
import numpy as np

In [41]:
def circuit_to_tableau(circuit: stim.Circuit) -> stim.Tableau:
    s = stim.TableauSimulator()
    s.do_circuit(circuit)
    return s.current_inverse_tableau() ** -1

In [42]:
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
coeffs = np.random.random(len(paulis))
from clapton.depolarization import GateGeneralDepolarizationModel

## Altering the Ansatz itself

In [43]:
rng = np.random.default_rng()

def add_pauli_twirl(circuit):

    TWIRL_GATES_CX = (
            (('I', 'I'), ('I', 'I')),
            (('I', 'X'), ('I', 'X')),
            (('I', 'Y'), ('Z', 'Y')),
            (('I', 'Z'), ('Z', 'Z')),
            (('X', 'I'), ('X', 'X')),
            (('X', 'X'), ('X', 'I')),
            (('X', 'Y'), ('Y', 'Z')),
            (('X', 'Z'), ('Y', 'Y')),
            (('Y', 'I'), ('Y', 'X')),
            (('Y', 'X'), ('Y', 'I')),
            (('Y', 'Y'), ('X', 'Z')),
            (('Y', 'Z'), ('X', 'Y')),
            (('Z', 'I'), ('Z', 'I')),
            (('Z', 'X'), ('Z', 'X')),
            (('Z', 'Y'), ('I', 'Y')),
            (('Z', 'Z'), ('I', 'Z')),
        )

    pauli_twirl_dict = {"I":0,"X":1,"Y":2,"Z":3}

    new_circuit = ParametrizedCliffordCircuit()
    for gate in circuit.gates:
        if gate.label == '2Q':
            control = gate.qbs[0]
            target = gate.qbs[1]

            (before0, before1), (after0, after1) = TWIRL_GATES_CX[
                rng.integers(len(TWIRL_GATES_CX))]

            new_circuit.PauliTwirl(control).fix(pauli_twirl_dict[before0])
            new_circuit.PauliTwirl(target).fix(pauli_twirl_dict[before1])
            new_circuit.Q2(control, target).fix(1)
            new_circuit.PauliTwirl(control).fix(pauli_twirl_dict[after0])
            new_circuit.PauliTwirl(target).fix(pauli_twirl_dict[after1])
        elif gate.label == "RY":
            new_circuit.RY(gate.qbs[0])
        elif gate.label == "RZ":
            new_circuit.RY(gate.qbs[0])

    return new_circuit

In [ ]:
# let's add a noise model where we specify global 1q and 2q gate errors

nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.1)
# nm = None

pauli_twirl = False

assert not pauli_twirl or nm is not None, "Depolarization model must be defined if Pauli Twirling is applied"

if pauli_twirl:

    init_ansatz = circular_ansatz_mirrored(N=len(paulis[0]), reps=1, fix_2q=True)
    vqe_pcirc = add_pauli_twirl(init_ansatz)
    # pauli_twirl_list = [twirled_circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True) for _ in range(100)]
    pauli_twirl_list = [add_pauli_twirl(init_ansatz) for _ in range(100)]
    vqe_pcirc.add_pauli_twirl_list(pauli_twirl_list)

    for i, circuit in enumerate(pauli_twirl_list):
        assert circuit_to_tableau(vqe_pcirc.stim_circuit()) == circuit_to_tableau(circuit.stim_circuit()), \
            f"Circuit Mismatch at index {i}"\

    vqe_pcirc.add_depolarization_model(nm)        
    pauli_twirl_list = [circuit.add_depolarization_model(nm) for circuit in pauli_twirl_list]

else: 
    vqe_pcirc = circular_ansatz_mirrored(N=len(paulis[0]), reps=1, fix_2q=True)
    vqe_pcirc.add_depolarization_model(nm)

In [ ]:
vqe_pcirc.number_parametrized_gates()

18

In [ ]:
vqe_pcirc.read()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
vqe_pcirc.stim_circuit().diagram()

q0: -I-DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.1)-@-DEPOLARIZE2(0.1)---I----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)----------------------------------------------------------@-DEPOLARIZE2(0.1)-X-DEPOLARIZE2(0.1)-I-DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------
                                                                                                                         | |                | |                                                                                                                                                                        | |                | |
q1: ---I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------|-|----------------X-DEPOLARIZE2(0.1)-@-DEPOLARIZE2(0.1)-I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------@-DEPOLARIZE2(0.1)-X-DEPOLARIZE2(0.1)-|-|------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------
                                                                                                                         | |                                   | |                                                                                                                                  | |                                   | |
q2: ----------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-@-DEPOLARIZE2(0.1)--------------------X-DEPOLARIZE2(0.1)--------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-X-DEPOLARIZE2(0.1)--------------------@-DEPOLARIZE2(0.1)----------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-

In [ ]:
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20,                        # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs
started GA at id 2 with 1 procs
started GA at id None with 1 procs




/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")



started GA at id 3 with 1 procs



/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evo

[0, array([1.13906681, 0.32491528, 0.81415153, 0.        ]), array([1, 3, 1, 0, 0, 1, 1, 2, 3, 2, 2, 1, 1, 1, 2, 0, 2, 2], dtype=object)]
[0, array([1.00250864, 0.34927517, 0.65323347, 0.        ]), array([2, 1, 0, 0, 0, 2, 3, 3, 0, 0, 1, 2, 0, 0, 3, 0, 1, 2], dtype=object)]
[0, array([-1.23144587, -0.41729433, -0.81415153,  0.        ]), array([3, 3, 1, 1, 2, 1, 1, 0, 2, 2, 3, 3, 3, 0, 2, 1, 2, 2], dtype=object)]
[0, array([-2.93657289, -0.9620712 , -1.97450169,  0.        ]), array([1, 1, 3, 3, 0, 1, 1, 3, 3, 2, 1, 3, 0, 0, 0, 1, 1, 1], dtype=object)]
[1, array([1.13295668, 0.31880515, 0.81415153, 0.        ]), array([1, 0, 0, 2, 0, 2, 1, 0, 2, 2, 2, 2, 2, 3, 3, 1, 1, 0], dtype=object)]
[1, array([0.98942916, 0.3361957 , 0.65323347, 0.        ]), array([2, 0, 3, 1, 2, 2, 3, 2, 2, 2, 1, 0, 1, 2, 2, 3, 3, 2], dtype=object)]
[1, array([-1.21725684, -0.4031053 , -0.81415153,  0.        ]), array([3, 2, 3, 2, 0, 1, 1, 1, 2, 2, 1, 3, 1, 2, 2, 2, 2, 0], dtype=object)]
[1, array([-2.91600471

In [49]:
ks_best

[0, 2, 1, 3, 2, 3, 1, 2, 2, 3, 2, 0, 1, 2, 2, 3, 2, 1]

In [50]:
energy_noisy, energy_noiseless

(np.float64(-0.36415103829932277), np.float64(-0.8141515328117074))

In [51]:
# differrence
np.abs(energy_noisy-energy_noiseless)

np.float64(0.4500004945123846)

In [ ]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.snapshot_noiseless().circ_snapshot_noiseless.diagram()

q0: -I------S_DAG-X-@-SQRT_Y-S_DAG-----@-X-SQRT_Y-S_DAG-
                  | |                  | |
q1: -Y------Z-----|-X-@------Y-----Z-@-X-|-Y------Z-----
                  |   |              |   |
q2: -SQRT_Y-S_DAG-@---X------Y-----I-X---@-Y------S-----